<center><h1> Hidden Markov Chain <h1/><center/>

Link to the article : https://medium.com/@soulawalid/hidden-markov-model-5c6bd4b867c4

In [1]:
import numpy as np

# Define states and observations
states = ['H', 'S']  # H for Happy, S for Sad
observations = ['G', 'B', 'R']  # G for Green, B for Blue, R for Red

# Define transition probabilities matrix (A)
transition_probabilities = np.array([
    [0.7, 0.3],  # From H to H, H to S
    [0.5, 0.5]   # From S to H, S to S
])

# Define emission probabilities matrix (B)
emission_probabilities = np.array([
    [0.1, 0.1, 0.8],  # H (Happy): P(G|H), P(B|H), P(R|H)
    [0.3, 0.5, 0.2]   # S (Sad): P(G|S), P(B|S), P(R|S)
])

# Define initial probabilities (pi)
initial_probabilities = np.array([0.4, 0.6])  # P(H) and P(S)

# Define the observation sequence (O)
observation_sequence = ['G', 'B', 'R']  # Observations over 3 days

# Create a mapping from observations to indices
obs_map = {obs: idx for idx, obs in enumerate(observations)}

# Print out the defined data
print("States:", states)
print("Observations:", observations)
print("Transition Probabilities:\n", transition_probabilities)
print("Emission Probabilities:\n", emission_probabilities)
print("Initial Probabilities:", initial_probabilities)
print("Observation Sequence:", observation_sequence)
print("Observation Mapping:", obs_map)


States: ['H', 'S']
Observations: ['G', 'B', 'R']
Transition Probabilities:
 [[0.7 0.3]
 [0.5 0.5]]
Emission Probabilities:
 [[0.1 0.1 0.8]
 [0.3 0.5 0.2]]
Initial Probabilities: [0.4 0.6]
Observation Sequence: ['G', 'B', 'R']
Observation Mapping: {'G': 0, 'B': 1, 'R': 2}


In [2]:
# Viterbi algorithm implementation
def viterbi_algorithm(obs_seq, states, start_prob, trans_prob, emis_prob, obs_map):
    n_states = len(states)
    T = len(obs_seq)
    
    # Initialize the Viterbi matrix and the path pointer matrix
    V = np.zeros((n_states, T))
    path = np.zeros((n_states, T), dtype=int)
    
    # Initialize base cases (t == 0)
    first_obs_idx = obs_map[obs_seq[0]]
    V[:, 0] = start_prob * emis_prob[:, first_obs_idx]
    
    # Run Viterbi for t > 0
    for t in range(1, T):
        obs_idx = obs_map[obs_seq[t]]
        for s in range(n_states):
            trans_prob_col = V[:, t-1] * trans_prob[:, s]
            max_prob_index = np.argmax(trans_prob_col)
            V[s, t] = trans_prob_col[max_prob_index] * emis_prob[s, obs_idx]
            path[s, t] = max_prob_index
    
    # Find the most probable final state
    final_state = np.argmax(V[:, T-1])
    prob_of_most_likely_sequence = V[final_state, T-1]
    
    # Backtrack to find the most probable state sequence
    most_likely_sequence = [final_state]
    for t in range(T-1, 0, -1):
        final_state = path[final_state, t]
        most_likely_sequence.append(final_state)
    most_likely_sequence.reverse()
    
    # Map state indices back to state names
    most_likely_sequence = [states[s] for s in most_likely_sequence]
    
    return most_likely_sequence, prob_of_most_likely_sequence

# Run the Viterbi algorithm
most_likely_sequence, prob_of_most_likely_sequence = viterbi_algorithm(
    observation_sequence, states, initial_probabilities,
    transition_probabilities, emission_probabilities, obs_map
)

print("Most likely sequence of moods:", most_likely_sequence)
print("Probability of the most likely sequence:", prob_of_most_likely_sequence)

Most likely sequence of moods: ['S', 'S', 'H']
Probability of the most likely sequence: 0.018
